In [45]:
cred_f = 'C:\Users\Vihaan\Documents\UCB\W210_Capstone\data_Crerdible\data\Credible'
ncred_f = 'C:\\Users\\Vihaan\\Documents\\UCB\\W210_Capstone\\data\\data\\notCredible'

In [46]:
import os
import json
import pandas as pd

modeling = pd.DataFrame(columns=('label', 'text', 'title'))
i = 0    
for root, dirs, files in os.walk(cred_f):
    for file in files:
        if file.endswith(".txt") and 'api' not in file:
             curr_file = os.path.join(root, file)
             #print curr_file
             with open(curr_file) as json_file:
                try:
                    data = json.load(json_file)
                    modeling.loc[i] = [0,data["text"],data["title"]]
                    #data["url"],data["date"],data["source"],data["text"],data["images"],data["videos"]
                    #print('')
                    i = i + 1
                except ValueError:
                    continue

for root, dirs, files in os.walk(ncred_f):
    for file in files:
        if file.endswith(".txt") and 'api' not in file:
             curr_file = os.path.join(root, file)
             #print curr_file
             with open(curr_file) as json_file:
                try:
                    data = json.load(json_file)
                    modeling.loc[i] = [1,data["text"],data["title"]]
                    #data["url"],data["date"],data["source"],data["text"],data["images"],data["videos"]
                    #print('')
                    i = i + 1
                except ValueError:
                    continue

In [47]:
modeling = modeling.sample(frac=1) #shuffle
modeling[1:5]
#print modeling.shape

,label,text,title
260,0.0,"As Inauguration Day approached, Obama White Ho...",Obama Administration Rushed to Preserve Intell...
707,1.0,MSNBC’s Mika Brzezinski just erased any linger...,MSNBC Just Admitted: “Our Job” Is To “Control ...
400,0.0,Republican efforts to revise the Affordable Ca...,House GOP proposal to replace Obamacare sparks...
621,1.0,By Kurt Nimmo\n\nThe French establishment warn...,France Blames Russia for Rise of Marine Le Pen...


In [54]:
#Split into train, test sets
from sklearn.cross_validation import train_test_split

train, test = train_test_split(modeling, test_size = 0.2)
print "train data shape is ", train.shape
print "test data shape is", test.shape
print "Fraction of not credible news in train data", round(train['label'].mean(),3)
print "Fraction of not credible news in test data", round(test['label'].mean(),3)

train data shape is  (815, 3)
test data shape is (204, 3)
Fraction of not credible news in train data 0.524
Fraction of not credible news in test data 0.529


In [55]:
#Build pipeline
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('classifier',  MultinomialNB()) ])

#pipeline.fit(train['text'].values, train['label'].values)
#pipeline.predict(test['text']) 

In [67]:
#Perform cross validation
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

k_fold = KFold(n=len(train), n_folds=6)
scores = []
f_scores=[]
confusion = numpy.array([[0, 0], [0, 0]])
for train_indices, test_indices in k_fold:
    train_text = train.iloc[train_indices]['text'].values
    train_y = train.iloc[train_indices]['label'].values
    #print train_text,train_y

    test_text = train.iloc[test_indices]['text'].values
    test_y = train.iloc[test_indices]['label'].values

    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    confusion += confusion_matrix(test_y, predictions)
    f_score = f1_score(test_y, predictions)
    score = accuracy_score(test_y, predictions)
    scores.append(score)
    f_scores.append(f_score)
    
print 'Cross Validation Metrics'
print('Total articles classified:', len(train))
print('Accuracy Score:', round(sum(scores)/len(scores),3))
print('F1 Score:', round(sum(f_scores)/len(f_scores),3))
print('Confusion matrix:')
print(confusion)


Cross Validation Metrics
('Total articles classified:', 815)
('Accuracy Score:', 0.926)
('F1 Score:', 0.926)
Confusion matrix:
[[374  14]
 [ 46 381]]


In [68]:
#Test on test dataset

test_preds = pipeline.predict(test['text'])
confusion = confusion_matrix(test['label'],test_preds)
f_score = f1_score(test['label'], test_preds)
score = accuracy_score(test['label'], test_preds)

print 'Test Dataset Metrics'
print('Total articles classified:', len(test))
print('Accuracy Score:', round(score,3))
print('F1 Score:', round(f_score,3))
print('Confusion matrix:')
print(confusion)

Test Dataset Metrics
('Total articles classified:', 204)
('Accuracy Score:', 0.922)
('F1 Score:', 0.922)
Confusion matrix:
[[94  2]
 [14 94]]
